In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

Drop columns (axis=1) that are not useful to train

In [ ]:
to_remove = ["job_name", "reason","channel","n_issues","ok_since","zip","gender"] 
df_train = pd.read_csv("../data/puzzle_train_dataset.csv").set_index("ids").drop(to_remove, axis=1)
df_test = pd.read_csv("../data/puzzle_test_dataset.csv").set_index("ids").drop(to_remove, axis=1)

In [4]:
print("Number of train samples: %d" %df_train.shape[0])
print("Number of test samples:  %d" %df_test.shape[0])
print("Number of features: %d" %(df_train.shape[1]-1))

Number of train samples: 64592
Number of test samples:  35000
Number of features: 18


Remove all rows where the default column has missing (NaN) values

In [5]:
df_train = df_train[pd.notnull(df_train['default'])]
df_train["default"] = df_train["default"].astype("int")

In [6]:
feats_info = pd.concat([df_train.isnull().sum(), df_train.dtypes, df_train.T.apply(lambda x: x.nunique(), axis=1)], axis=1)
feats_info.columns = ["Missing","Type","Unique"]
print("\n Dataset Information (missing features, data type, unique values)")
print(feats_info)


 Dataset Information (missing features, data type, unique values)
                    Missing     Type  Unique
default                   0    int64       2
score_1                   0      str       7
score_2                   0      str      35
score_3                   0  float64      87
score_4                   0  float64   59966
score_5                   0  float64   59966
score_6                   0  float64   59966
risk_rate                 0  float64      81
amount_borrowed           0  float64   50484
borrowed_in_months        0  float64       2
credit_limit          18779  float64   26238
income                    0  float64   54273
sign                  18938      str      12
facebook_profile       5971   object       2
state                     0      str      50
real_state                0      str       5
n_bankruptcies          202  float64       7
n_defaulted_loans        18  float64       5
n_accounts                0  float64      45
